In [30]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from pathlib import Path
import itertools
import sklearn
import sklearn.metrics
import io
import shutil


In [31]:
BATCH_SIZE = 16
IMG_HEIGHT, IMG_WIDTH = (224, 224)
PREPROCESS_SEED = 123
CHECKPOINTS_DIR = Path("checkpoints")
LOGS_DIR = Path("logs")

In [32]:
base_data_dir = Path("..", "..", "input", "arch-recognizer-dataset").absolute()
val_data_dir = base_data_dir / "val"
test_data_dir = base_data_dir / "test"
train_data_dir = base_data_dir / "train"

In [33]:
# # Import data

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     val_data_dir,
#     labels="inferred",
#     label_mode="int",
#     seed=PREPROCESS_SEED,
#     image_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     crop_to_aspect_ratio=True,
# )

# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     test_data_dir,
#     labels="inferred",
#     label_mode="int",
#     seed=PREPROCESS_SEED,
#     image_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     crop_to_aspect_ratio=True,
# )

# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     train_data_dir,
#     labels="inferred",
#     label_mode="int",
#     seed=PREPROCESS_SEED,
#     image_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     crop_to_aspect_ratio=True,
# )

# class_names = train_ds.class_names


# def preprocess(img):
#     # tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(img)
#     # tf.keras.applications.resnet.preprocess_input(img)
#     return img


# train_ds.map(lambda img, _: preprocess(img))
# val_ds.map(lambda img, _: preprocess(img))
# test_ds.map(lambda img, _: preprocess(img))

# train_ds = train_ds.shuffle(5000).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
# val_ds = val_ds.shuffle(5000).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
# test_ds = test_ds.shuffle(5000).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Found 2021 files belonging to 25 classes.
Found 1012 files belonging to 25 classes.
Found 7080 files belonging to 25 classes.


In [35]:
# Define functions to generate confusion matrix

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(len(class_names), len(class_names)))
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45, ha="right")
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype("float") / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    return figure

In [36]:
# Define function for restoring checkpoints 
def restore_weights_from_checkpoint(model):
    latest_cp = tf.train.latest_checkpoint(CHECKPOINTS_DIR)
    if latest_cp:
        model.load_weights(latest_cp)
        _, restored_test_acc = model.evaluate(test_ds, verbose=2)
        print(f"Restored model test accuracy: {restored_test_acc}")
    return model


In [34]:
# Configure Hyperparameters

CNN_APPS = {
    tf.keras.applications.VGG19.__name__: {
        "image_size": (224, 224),
        "scale": 1.0 / 255,
        "offset": 0,
        "preprocessor": tf.keras.applications.vgg19.preprocess_input,
    },
    tf.keras.applications.ResNet50V2.__name__: {
        "image_size": None,
        "scale": 1.0 / 255,
        "offset": 0,
        "preprocessor": tf.keras.applications.resnet.preprocess_input,
    },
    tf.keras.applications.ResNet152V2.__name__: {
        "image_size": None,
        "scale": 1.0 / 255,
        "offset": 0,
        "preprocessor": tf.keras.applications.resnet.preprocess_input,
    },
    tf.keras.applications.InceptionV3.__name__: {
        "image_size": None,
        "scale": 1.0,
        "offset": 0,
        "preprocessor": tf.keras.applications.inception_v3.preprocess_input,
    },
    tf.keras.applications.InceptionResNetV2.__name__: {
        "image_size": None,
        "scale": 1.0,
        "offset": 0,
        "preprocessor": tf.keras.applications.inception_resnet_v2.preprocess_input,
    },
    tf.keras.applications.MobileNetV2.__name__: {
        "image_size": (224, 224),
        # "scale": 1.0 / 127.5,
        "scale": 1.0,
        # "offset": -1,
        "offset":0,
        "preprocessor": tf.keras.applications.mobilenet_v2.preprocess_input,
    },
    tf.keras.applications.DenseNet201.__name__: {
        "image_size": (224, 224),
        "scale": 1.0 / 255,
        "offset": 0,
        "preprocessor": tf.keras.applications.densenet.preprocess_input,
    },
    tf.keras.applications.EfficientNetB7.__name__: {
        "image_size": None,
        "scale": 1.0 / 255,
        "offset": 0,
        "preprocessor": tf.keras.applications.efficientnet.preprocess_input,
    },
}

HP_CNN_MODEL = hp.HParam("pooling", hp.Discrete(list(CNN_APPS.keys())))
HP_WEIGHTS = hp.HParam("pooling", hp.Discrete(["", "imagenet"]))
HP_LEARNING_RATE = hp.HParam("learning_rate", hp.RealInterval([5e-5, 1e-4, 3e-4]))

METRIC_ACCURACY = "accuracy"

with tf.summary.create_file_writer(f"logs/hparam_tuning").as_default():
    hp.hparams_config(
        hparams=[HP_CNN_MODEL, HP_WEIGHTS, HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")],
    )


In [40]:
print(tf.keras.applications.ResNet50.__name__)

ResNet50


In [ ]:
# Define training run routine
# Import data

def get_datasets(img_height, img_width):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_data_dir,
        labels="inferred",
        label_mode="int",
        seed=PREPROCESS_SEED,
        image_size=(img_height, img_width),
        batch_size=BATCH_SIZE,
        shuffle=True,
        crop_to_aspect_ratio=True,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        val_data_dir,
        labels="inferred",
        label_mode="int",
        seed=PREPROCESS_SEED,
        image_size=(img_height, img_width),
        batch_size=BATCH_SIZE,
        shuffle=True,
        crop_to_aspect_ratio=True,
    )
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_data_dir,
        labels="inferred",
        label_mode="int",
        seed=PREPROCESS_SEED,
        image_size=(img_height, img_width),
        batch_size=BATCH_SIZE,
        shuffle=True,
        crop_to_aspect_ratio=True,
    )
    return train_ds, val_ds, test_ds

# class_names = train_ds.class_names




def run(hparams, run_name):
    cnn_app = CNN_APPS[[hparams[HP_CNN_MODEL]]]

    train_ds, val_ds, test_ds = get_datasets(*cnn_app["image_size"])
    class_names = train_ds.class_names

    train_ds.map(lambda img, _: cnn_app["preprocessor"](img))
    val_ds.map(lambda img, _: cnn_app["preprocessor"](img))
    test_ds.map(lambda img, _: cnn_app["preprocessor"](img))

    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    def get_cnn_model(hparams):
        return {
            tf.keras.applications.ResNet50.__name__: tf.keras.applications.ResNet50(
                include_top=True,
                weights=hparams[HP_WEIGHTS] if hparams[HP_WEIGHTS] else None,
                classes=len(class_names) if not hparams[HP_WEIGHTS] else None,
            ),
    }[hparams[HP_CNN_MODEL]]

    model = restore_weights_from_checkpoint(
        tf.keras.models.Sequential(
            [
                # Augmentation
                tf.keras.layers.experimental.preprocessing.RandomFlip(
                    "horizontal", input_shape=(*cnn_app["image_size"], 3)
                ),
                tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
                # Convolution
                get_cnn_model(hparams),
            ]
        )
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE]),
        loss="sparse_categorical_crossentropy",
        metrics=[METRIC_ACCURACY],
    )

    # Defining a file writer for confusion matrix logging
    cm_file_writer = tf.summary.create_file_writer(str(LOGS_DIR / "cm"))

    def log_confusion_matrix(epoch, logs):
        pred_y, true_y = [], []
        for batch_X, batch_y in test_ds:
            true_y.extend(batch_y)
            pred_y.extend(np.argmax(model.predict(batch_X), axis=-1))
        cm_data = np.nan_to_num(sklearn.metrics.confusion_matrix(true_y, pred_y))
        cm_figure = plot_confusion_matrix(cm_data, class_names=class_names)
        cm_image = plot_to_image(cm_figure)
        with cm_file_writer.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,
        callbacks=[
            tf.keras.callbacks.TensorBoard(
                log_dir=LOGS_DIR / run_name, histogram_freq=1, profile_batch=1
            ),
            tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=str(CHECKPOINTS_DIR)
                + f"/{run_name}"
                + "-epoch-{epoch:04d}.ckpt",
                monitor=METRIC_ACCURACY,
                verbose=1,
                save_best_only=True,
                save_freq="epoch",
            ),
            tf.keras.callbacks.EarlyStopping(
                min_delta=0.0001, patience=10, restore_best_weights=True
            ),
        ],
    )

    _, accuracy = model.evaluate(test_ds)
    return accuracy


In [38]:
# Execute training

# If no saved checkpoints, reset logs by deleting the logs dir
if not CHECKPOINTS_DIR.exists() or not list(CHECKPOINTS_DIR.iterdir()):
    shutil.rmtree(LOGS_DIR)

# # Load the TensorBoard notebook extension
# %reload_ext tensorboard
# %tensorboard --logdir logs --bind_all --reload_interval 10

# Perform training runs
run_num = 0
for cnn_model in HP_CNN_MODEL.domain.values:
    for weights in HP_WEIGHTS.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams = {
                HP_CNN_MODEL: cnn_model,
                HP_WEIGHTS: weights,
                HP_LEARNING_RATE: learning_rate,
            }
            run_name = f"run-{run_num}-{cnn_model}-{weights}-{learning_rate}"
            run_logs_dir = LOGS_DIR / run_name
            print(f"--- Starting trial: run-{run_num}")
            print({h.name: hparams[h] for h in hparams})
            with tf.summary.create_file_writer(logdir=LOGS_DIR / run_name).as_default():
                hp.hparams(hparams)  # record the values used in this run
                test_accuracy = run(hparams, run_name)
                tf.summary.scalar(METRIC_ACCURACY, test_accuracy, step=1)
            run_num += 1


Reusing TensorBoard on port 6006 (pid 43022), started 0:02:39 ago. (Use '!kill 43022' to kill it.)

--- Starting trial: run-0
{'pooling': 'avg', 'optimizer': 'adam'}
Epoch 1/100
443/443 [==============================] - 116s 230ms/step - loss: 3.2082 - accuracy: 0.1177 - val_loss: 3.2994 - val_accuracy: 0.1128

Epoch 00001: accuracy improved from -inf to 0.11766, saving model to checkpoints/run-0-epoch-0001.ckpt
INFO:tensorflow:Assets written to: checkpoints/run-0-epoch-0001.ckpt/assets


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
285/443 [==================>...........] - ETA: 31s - loss: 2.9523 - accuracy: 0.1320

KeyboardInterrupt: 